In [172]:
#importing necessary libraries
import numpy as np
import pandas as pd
import pickle
import requests
import json
import logging


In [173]:
#acessing our data base from token and Keysapce.
#loading Database
from astrapy import DataAPIClient

# Initialize the client
client = DataAPIClient("AstraCS:DNFtgYBgkWOvLusnvtBJhClI:65cf4a05b893b58375a388761136a2f5827cb60a5288b562c3651afd098d5225")

db = client.get_database_by_api_endpoint(
  "https://f19f2881-aa0a-4909-be2b-fbdb67f0ae3e-us-east1.apps.astra.datastax.com",
    keyspace="keyspace",
)
      
print(f"Connected to Astra DB: {db.list_collection_names()}")

Connected to Astra DB: ['investment_data']


In [174]:
# Retrieve the collection (table) within the database
collection = db["investment_data"]

# Query the data from the 'investment_data' table
result = collection.find()

# Convert the result to a pandas DataFrame
df = pd.DataFrame(list(result))

# Save it to a CSV file
df.to_csv('output.csv', index=False)

# Drop unnecessary columns
df = df.drop(columns=['_id', 'Date'])
# Display the DataFrame
df.head()


,Open,High,Low,Close,Adj_Close,Volume
0,633.200012,639.419983,626.780029,627.039978,627.039978,2862500
1,378.269989,382.660004,356.619995,359.700012,359.700012,12684000
2,289.100006,298.850006,285.649994,288.850006,288.850006,11444800
3,273.630005,290.309998,271.220001,288.940002,288.940002,12914000
4,517,519.570007,509.769989,512.400024,512.400024,1673900


In [175]:
# Convert columns to appropriate data types
df['Open'] = pd.to_numeric(df['Open'], errors='coerce')
df['High'] = pd.to_numeric(df['High'], errors='coerce')
df['Low'] = pd.to_numeric(df['Low'], errors='coerce')
df['Volume'] = pd.to_numeric(df['Volume'], errors='coerce')
df['Close'] = pd.to_numeric(df['Close'], errors='coerce')
df['Adj_Close'] = pd.to_numeric(df['Adj_Close'], errors='coerce')

# Verify data types
print(df.dtypes)


Open         float64
High         float64
Low          float64
Close        float64
Adj_Close    float64
Volume         int64
dtype: object


In [176]:
df = df.dropna(subset=['Open', 'High', 'Low', 'Volume', 'Adj_Close'])  
# Drop rows with NaN in any of the key columns


In [177]:
df.shape

(1009, 6)

In [178]:
df.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj_Close', 'Volume'], dtype='object')

In [179]:
# Drop rows with NaN values in any of the important columns
df = df.dropna(subset=['Open', 'High', 'Low', 'Volume', 'Adj_Close'])

# Or, alternatively, you can fill missing values with the mean
df['Open'].fillna(df['Open'].mean(), inplace=True)
df['High'].fillna(df['High'].mean(), inplace=True)
df['Low'].fillna(df['Low'].mean(), inplace=True)
df['Volume'].fillna(df['Volume'].mean(), inplace=True)
df['Adj_Close'].fillna(df['Adj_Close'].mean(), inplace=True)


C:\Users\asus\AppData\Local\Temp\ipykernel_16956\3236645071.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Open'].fillna(df['Open'].mean(), inplace=True)
C:\Users\asus\AppData\Local\Temp\ipykernel_16956\3236645071.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exa

In [180]:
x = df[['Open', 'High', 'Low', 'Volume']]
y = df['Adj_Close']# Target variable

In [181]:
print(f"Final x shape: {x.shape}, y shape: {y.shape}")


Final x shape: (1009, 4), y shape: (1009,)


In [182]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size =0.25)
print(len(x_train))
print(len(x_test))
print(len(y_train))
print(len(y_test))

756
253
756
253


In [183]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
len(x_test)

253

In [184]:
from sklearn.metrics import *
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

# Display the evaluation metrics
print(f"R-squared Score: {r2}")
print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")

R-squared Score: 0.9989992648153723
Mean Squared Error: 11.233439554470769
Mean Absolute Error: 2.661179134868696


In [185]:
pickle.dump(model,open('model.pkl','wb'))
model=pickle.load(open('model.pkl','rb'))

In [186]:
# Convert the predictions to a DataFrame and save them to a CSV file
predictions_df = pd.DataFrame({
    'Actual': y_test,
    'Predicted': y_pred
})
predictions_df.to_csv('predictions.csv', index=False)

# Display the predictions
predictions_df.head()


,Actual,Predicted
971,353.160004,356.583035
157,506.519989,506.360040
5,447.670013,447.566207
901,329.600006,328.916675
640,310.829987,309.770057


In [187]:
# Make sure the columns in x_test are the same as in x_train
x_test = x_test[x_train.columns]

# Predict
y_pred = model.predict(x_test)


In [188]:
# If you're passing a NumPy array to the model, convert it to a DataFrame:
x_test = pd.DataFrame(x_test, columns=x_train.columns)
y_pred = model.predict(x_test)
